### Knock41: 基本的な Directory を生成しよう
> #### Master data と input data
> 明確な基準はないが、更新頻度によって仕訳するとよい。
> - 更新頻度-高: input data
> - 更新頻度-低: master data

In [70]:
# Directory path の定義
import os
data_dir = 'data'
input_dir = os.path.join(data_dir, '0_input')
output_dir = os.path.join(data_dir, '10_output')
master_dir = os.path.join(data_dir, '99_master_dir')
print(input_dir)

data\0_input


`os.mkdir()` と `os.makedirs()` の違い
- `os.makedirs()` は、再帰的に中間の Directory を自動で生成してくれる。

In [71]:
# Directory の生成
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)
os.makedirs(master_dir, exist_ok=True)

### Knock42: 入力 Data の check機構を作ろう

In [72]:
# Master data の読み込み
import pandas as pd

m_area_file = 'm_area.csv'
m_store_file = 'm_store.csv'
m_area = pd.read_csv(os.path.join(master_dir, m_area_file))
m_store = pd.read_csv(os.path.join(master_dir, m_store_file))
m_area.head(3)

,area_cd,wide_area,narrow_area
0,TK,東京,東京
1,KN,神奈川,神奈川
2,CH,千葉,千葉


In [73]:
# 直接 File 名を指定せずに、変数を対象年月として File を定義
tg_ym = '202007'
target_file = 'tbl_order_' + tg_ym + '.csv'
target_data = pd.read_csv(os.path.join(input_dir, target_file))

- なるべく混乱を避けるために、File 名ではなく、年月を指定するようにする等の工夫を行なう。
- 毎月特定営業日に Data 更新を行なうなどが決まっている場合は、`datetime` 等で現在の年月を取得するのも良い。
- Data がない, File 名が違う場合, `FileNotFoundError` 等が発生し Program が止まるため間違いを確認できる。
- Error も起きずに自分が気づかないうちに意図しない処理が行なわれてしまい、間違った Report を配ってしまうことが一番、問題となる。
- 毎月変化していく Data は、必ず check 機構を取り入れる。
- Data の中身を出力することが Check 機構の基本だが Human error を起こす場合がある。
- 可能な限り、Error を意図的に発生させて、Program を Stop できる機構を入れることが望ましい。

In [74]:
# Data check 機構（正常動作時）
import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_data = pd.to_datetime(target_data['order_accept_date']).min()
max_str_date = max_date.strftime('%Y%m')
min_str_date = min_data.strftime('%Y%m')
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

日付が一致しました


csv や Excel などのような Data は簡単に File 名を変更できてしまうため、Data の内容から check できる機構を入れておくことで間違いに気が付ける。

In [75]:
# Data check 機構（Error 動作時）
import datetime

max_date = pd.to_datetime(target_data['order_accept_date']).max()
min_data = pd.to_datetime(target_data['order_accept_date']).min()
max_str_date = max_date.strftime('%Y%m')
min_str_date = min_data.strftime('%Y%m')
if tg_ym == min_str_date and tg_ym == max_str_date:
    print("日付が一致しました")
else:
    raise Exception("日付が一致しません")

日付が一致しました


Check 機構は、取り組んでいる Data によって違いが出てくる。
どういった規則で check を行なうべきか考えて構築する。

In [76]:
# DataFrame の各種初期化処理
def calc_delta(t):
    t1, t2 = t
    delta = t2 - t1
    return delta.total_seconds() / 60


def init_tran_df(trg_df):
    # 保守用店舗 Data の削除
    trg_df = trg_df.loc[trg_df['store_id'] != 999]

    trg_df = pd.merge(trg_df, m_store, on='store_id', how='left')
    trg_df = pd.merge(trg_df, m_area, on='area_cd', how='left')

    # Master にない Code に対応した文字列を設定
    trg_df.loc[trg_df['takeout_flag'] == 0, 'takeout_name'] = "デリバリー"

    trg_df.loc[trg_df['takeout_flag'] == 1, 'takeout_name'] = "お持ち帰り"

    trg_df.loc[trg_df['status'] == 0, 'status_name'] = "受付"
    trg_df.loc[trg_df['status'] == 1, 'status_name'] = "お支払済"
    trg_df.loc[trg_df['status'] == 2, 'status_name'] = "お渡し済"
    trg_df.loc[trg_df['status'] == 9, 'status_name'] = "キャンセル"

    trg_df.loc[:, 'order_date'] = pd.to_datetime(trg_df['order_accept_date']).dt.date

    # 配達までの時間を計算
    trg_df.loc[:, 'order_accept_datetime'] = pd.to_datetime(trg_df['order_accept_date'])
    trg_df.loc[:, 'delivered_datetime'] = pd.to_datetime(trg_df['delivered_date'])
    trg_df.loc[:, 'delta'] = trg_df[['order_accept_datetime', 'delivered_datetime']].apply(calc_delta, axis=1)

    return trg_df


# 当月分を初期化
target_data = init_tran_df(target_data)

### Knock43: Reporting（本部向け）を関数化してみよう
Reporting などの出力機能等は関数として保持しておくことで
- 可読性の工場
- Report 内容を変更する際に Program の変更箇所が一目でわかるようなる

という Merit がある。

In [77]:
# Excel の Library import と店舗売上 Ranking の集計関数
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill, Border, Side, Font


def get_rank_df(target_data):
    # 店舗別の Data 作成、Ranking DF の返却
    tmp = target_data.loc[target_data['status'].isin([1, 2])]
    rank = tmp.groupby(['store_id'])['total_amount'].sum().sort_values(ascending=False)
    rank = pd.merge(rank, m_store, on='store_id', how='left')

    return rank

In [78]:
# Cancel 率の Ranking 集計関数
def get_cancel_rank_df(target_data):
    # Cancel 率の計算、Ranking DF の返却
    cancel_df = pd.DataFrame()
    cancel_cnt = target_data.loc[target_data['status'] == 9].groupby(['store_id'])['store_id'].count()
    order_cnt = target_data.loc[target_data['status'].isin([1, 2, 9])].groupby(['store_id'])['store_id'].count()
    cancel_rate = (cancel_cnt / order_cnt) * 100
    cancel_df['cancel_rate'] = cancel_rate
    cancel_df = pd.merge(cancel_df, m_store, on='store_id', how='left')
    cancel_df = cancel_df.sort_values('cancel_rate', ascending=True)

    return cancel_df

In [79]:
# Data の出力処理
def data_export(df, ws, row_start, col_start):
    # Style 定義
    side = Side(style='thin', color='008080')
    border = Border(top=side, bottom=side, left=side, right=side)

    rows = dataframe_to_rows(df, index=False, header=True)

    for row_no, row in enumerate(rows, row_start):
        for col_no, value in enumerate(row, col_start):
            cell = ws.cell(row_no, col_no)
            cell.value = value
            cell.border = border
            if row_no == row_start:
                cell.fill = PatternFill(patternType='solid', fgColor='008080')
                cell.font = Font(bold=True, color='FFFFFF')

In [80]:
# 本社向け Reporting data 処理
def make_report_hq(target_data, output_folder):
    rank = get_rank_df(target_data)
    cancel_rank = get_cancel_rank_df(target_data)

    # Excel 出力処理
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Summary-Report（本社向け）"

    cell = ws.cell(1, 1)
    cell.value = f"本社向け {max_str_date} 月度 Summary Report"
    cell.font = Font(bold=True, color='008080', size=20)

    cell = ws.cell(3, 2)
    cell.value = f"{max_str_date}月度 売上総額"
    cell.font = Font(bold=True, color='008080', size=20)

    cell = ws.cell(3, 6)
    cell.value = f"{'{:,}'.format(rank['total_amount'].sum())}"
    cell.font = Font(bold=True, color='008080', size=20)

    # 売上 Ranking を直接出力
    cell = ws.cell(5, 2)
    cell.value = f"売上 Ranking"
    cell.font = Font(bold=True, color='008080', size=16)

    # 表の貼り付け
    data_export(rank, ws, 6, 2)

    # Cancel率 Ranking を直接出力
    cell = ws.cell(5, 8)
    cell.value = f"Cancel 率 Ranking"
    cell.font = Font(bold=True, color='008080', size=16)

    # 表の貼り付け位置
    data_export(cancel_rank, ws, 6, 8)

    wb.save(os.path.join(output_folder, f"report_hq_{max_str_date}.xlsx"))
    wb.close()

関数化（構造化）は、可読性や Maintenance 性も向上するので、なるべく意識して関数を作るように心がける。
※処理を細かくし過ぎないように注意する。

### Knock44: Reporting（店舗向け）を関数化してみよう


In [81]:
# 店舗の売上 Ranking と店舗の売上集計関数
def get_store_rank(target_id, target_df):
    rank = get_rank_df(target_df)
    store_rank = rank.loc[rank['store_id'] == target_id].index + 1

    return store_rank[0]


def get_store_sale(target_id, target_df):
    rank = get_rank_df(target_df)
    store_sale = rank.loc[rank['store_id'] == target_id]['total_amount']

    return store_sale

関数を分けておくと、再利用が可能になり、効率も良くなり、不具合発生率も低くなる。

In [82]:
# 店舗単位の Cancel 率 Rank, Cancel 数の集計関数
def get_store_cancel_rank(target_id, target_df):
    cancel_df = get_cancel_rank_df(target_df)
    cancel_df = cancel_df.reset_index()
    store_cancel_rank = cancel_df.loc[cancel_df['store_id'] == target_id].index + 1

    return store_cancel_rank[0]


def get_store_cancel_count(target_id, target_df):
    store_cancel_count = target_df.loc[
        (target_df['status'] == 9)
        & (target_df['store_id'] == target_id)
        ].groupby(['store_id'])['store_id'].count()
    return store_cancel_count

In [83]:
# 店舗毎の配達までの時間 Ranking と集計関数
def get_delivery_rank_df(target_id, target_df):
    delivery = target_df.loc[target_df['status'] == 2]
    delivery_rank = delivery.groupby(['store_id'])['delta'].mean().sort_values()
    delivery_rank = pd.merge(delivery_rank, m_store, on='store_id', how='left')

    return delivery_rank


def get_delivery_rank_store(target_id, target_df):
    delivery_rank = get_delivery_rank_df(target_id, target_df)
    store_delivery_rank = delivery_rank.loc[delivery_rank['store_id'] == target_id].index + 1

    return store_delivery_rank[0]

In [87]:
# 店舗個別の Reporting 出力関数

# 店舗向け Reporting Data 処理
def make_report_store(target_data, target_id, output_folder):
    rank = get_store_rank(target_id, target_data)
    sale = get_store_sale(target_id, target_data)
    cancel_rank = get_store_cancel_rank(target_id, target_data)
    cancel_count = get_store_cancel_count(target_id, target_data)
    delivery_df = get_delivery_rank_df(target_id, target_data)
    delivery_rank = get_delivery_rank_store(target_id, target_data)

    store_name = m_store.loc[m_store['store_id'] == target_id]['store_name'].values[0]

    # Excel 出力処理
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "店舗向け Reporting"

    cell = ws.cell(1, 1)
    cell.value = f"{store_name} {max_str_date}月度 Summary Report"
    cell.font = Font(bold=True, color='008080', size=20)

    cell = ws.cell(3, 2)
    cell.value = f"{max_str_date}月度 売上総額"
    cell.font = Font(bold=True, color='008080', size=20)

    cell = ws.cell(3, 6)
    cell.value = f"{'{:,}'.format(sale.values[0])}"
    cell.font = Font(bold=True, color='080080', size=20)

    # 売上 Ranking を直接出力
    cell = ws.cell(5, 2)
    cell.value = f"売上 Ranking"
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(5, 5)
    cell.value = f"{rank}位"
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(6, 2)
    cell.value = f"売上 Data"
    cell.font = Font(bold=True, color='008080', size=16)

    # 表の貼り付け
    tmp_df = target_data.loc[
        (target_data['store_id'] == target_id)
        & (target_data['status'].isin([1, 2]))
        ]
    tmp_df = tmp_df[['order_accept_date', 'customer_id', 'total_amount', 'takeout_name', 'status_name']]
    data_export(tmp_df, ws, 7, 2)

    # Cancel 率の Ranking を直接出力
    cell = ws.cell(5, 8)
    cell.value = f"Cancel 率 Ranking"
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(5, 12)
    cell.value = f"{cancel_rank}位 {cancel_count.values[0]}回"
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(6, 8)
    cell.value = f"Cancel Data"
    cell.font = Font(bold=True, color='008080', size=16)

    # 表の貼り付け
    tmp_df = target_data.loc[
        (target_data['store_id'] == target_id)
        & (target_data['status'] == 9)
        ]
    tmp_df = tmp_df[['order_accept_date', 'customer_id', 'total_amount', 'takeout_name', 'status_name']]
    data_export(tmp_df, ws, 7, 8)

    # 配達完了までの時間を直接出力
    ave_time = delivery_df.loc[delivery_df['store_id'] == target_id]['delta'].values[0]
    cell = ws.cell(5, 14)
    cell.value = f"配達完了までの時間 Ranking"
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(5, 18)
    cell.value = f"{delivery_rank}位 平均{ave_time}分"
    cell.font = Font(bold=True, color='008080', size=16)

    cell = ws.cell(6, 14)
    cell.value = f"各店舗の配達時間 Rankingi"
    cell.font = Font(bold=True, color='008080', size=16)

    # 表の貼り付け
    data_export(delivery_df, ws, 7, 14)

    wb.save(os.path.join(output_folder, f"{target_id}_{store_name}_report_{max_str_date}.xlsx"))
    wb.close()

### Knock45: 関数を実行し動作を確認してみよう

In [85]:
# 本部向け Report
make_report_hq(target_data, output_dir)

In [88]:
# 各店舗向け Repot（全店舗実施）
for store_id in m_store.loc[m_store['store_id'] != 999]['store_id']:
    make_report_store(target_data, store_id, output_dir)